In [1]:
'''
这一栏是为了拿维州的全部postcodes
'''
import pandas as pd
# Defining of the dataframe
# built-in imports
import re
from json import dump

from collections import defaultdict

# user packages
from bs4 import BeautifulSoup, SoupStrainer
from urllib.request import urlopen
import requests
df = pd.DataFrame(columns=['postcode'])
headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}
table = BeautifulSoup(requests.get('https://www.worldpostalcodes.org/l1/en/au/australia/list/r1/list-of-postcodes-in-victoria',headers = headers).text, "html.parser")
# Collecting Ddata
for row in table.find_all('tr'):    
    # Find all data for each column
    columns = row.find_all('td')
    
    if(columns != []):
        postcode = columns[0].text.strip()

        df = df.append({'postcode':postcode}, ignore_index=True)    
postcodes = df['postcode'].tolist()
postcodes = [int(x) for x in postcodes if x]

/var/folders/7q/psdflw4d7cdc_6hv6hqsr59m0000gn/T/ipykernel_2402/631642778.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'postcode':postcode}, ignore_index=True)
/var/folders/7q/psdflw4d7cdc_6hv6hqsr59m0000gn/T/ipykernel_2402/631642778.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'postcode':postcode}, ignore_index=True)
/var/folders/7q/psdflw4d7cdc_6hv6hqsr59m0000gn/T/ipykernel_2402/631642778.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'postcode':postcode}, ignore_index=True)
/var/folders/7q/psdflw4d7cdc_6hv6hqsr59m0000gn/T/ipykernel_2402/631642778.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

In [2]:
'''
这个东西是所有需要爬取信息的网站
'''
url_links = pd.read_csv("../data/raw/scrapping_url.csv",names=["links", "postcode"])

In [3]:
user_agents =  ['Mozilla/5.0 (Windows NT 6.1; rv:2.0.1) Gecko/20100101 Firefox/4.0.1',
                'Mozilla/5.0 (Windows; U; Windows NT 6.1; en-us) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50',
                'Opera/9.80 (Windows NT 6.1; U; en) Presto/2.8.131 Version/11.11',
               'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36',
               'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 6.1; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; InfoPath.3)'] 

In [4]:
# funtion 更改当前被ban的ip
'''
网上的ip貌似链接不到我们要爬的网站，所以换ip的这个function暂时搁置
'''
def changeIP(current_ip):
    url = "https://free-proxy-list.net/"
    headers = {'User-Agent':random.choice(user_agents)} 
    bs_object = BeautifulSoup(requests.get(url,headers = headers).text, "html.parser") 
    for tr in bs_object.find("tbody").findAll("tr"):
        ip_information = tr.findAll("td")
        proxy = ip_information[0].text + ":" + ip_information[1].text
        if ip_information[4].text == "transparent" and proxy != current_ip:
            break
    return proxy

In [5]:
'''
这里是正式开始爬取信息的地方，然后3168以前的邮编已经爬过了，可以尝试爬后面的
'''
import random
from collections import defaultdict
import time

 
property_metadata = defaultdict(dict)
# local_ip = '45.79.208.64:44554'
# current_ip = changeIP(local_ip) 

for i in range(852,1252):
    # sleep for 0-1 seconds
    time.sleep(random.random())
    
    headers = {'User-Agent':random.choice(user_agents)}  
    
   
    bs_object = BeautifulSoup(requests.get(url_links.iloc[i]["links"],headers = headers).text, "html.parser")
    
    '''
    # if 403 happened, changed current ip address 
    while bs_object.find("head").find("title").text == "403 Forbidden":
        current_ip = changeIP(current_ip)
        bs_object = BeautifulSoup(requests.get(row["links"],headers = headers, proxies={"http": current_ip, "https": current_ip}).text, "html.parser")
    '''    
        
    postcodes = url_links.iloc[i]["postcode"]
    bs_element = []
    if bs_object.find_all('div', {'class':'property odd clearfix'}) is not None: 
        bs_element.append(bs_object.find_all('div', {'class':'property odd clearfix'}))
    
    if bs_object.find_all('div', {'class':'property even clearfix'}) is not None:
        bs_element.append(bs_object.find_all('div', {'class':'property even clearfix'}))
    
    if len(bs_element) > 0:    
        for bs in bs_element:     
            for prop in bs:
                # looks for the header class to get property name
                name = prop \
                    .find("h2", {"class": "address"}) \
                    .text
                property_metadata[name]['name'] = prop \
                    .find("h2", {"class": "address"}) \
                    .text
                property_metadata[name]['postcode'] = postcodes

                cost = []     
                for li in prop.find("section", {"class": "grid-100 historical-price"}).find("ul"):              
                    cost_text = li.text
                    cost.append(cost_text)
                if " " in cost:
                    cost.remove(" ")
                property_metadata[name]['cost_text'] = cost

                if prop.find("p",{"class":"property-meta type"}) is not None:
                    property_metadata[name]['property_type'] = prop.find("p",{"class":"property-meta type"}).text

                if prop.find("p",{"class":"property-meta bed"}) is not None:
                    property_metadata[name]['bed'] = prop.find("p",{"class":"property-meta bed"}).text

                if prop.find("p",{"class":"property-meta bath"}) is not None:
                    property_metadata[name]['bath'] = prop.find("p",{"class":"property-meta bath"}).text

                if prop.find("p",{"class":"property-meta car"}) is not None:
                    property_metadata[name]['car'] = prop.find("p",{"class":"property-meta car"}).text

                property_metadata[name]['latitude'] = prop["data-lat"]  
                property_metadata[name]['longitude'] = prop["data-lng"]

# output to csv
pd.DataFrame(property_metadata).to_csv("../data/raw/row_852_1252.csv")

In [6]:
a = pd.read_csv('../data/raw/row_852_1252.csv')
a=a.T
a = a.reset_index(drop=True)
print(a)

                                                      0         1  \
0                                                  name  postcode   
1      2/8 DENNEYS STREET WARRNAMBOOL, WARRNAMBOOL WEST      3280   
2        21 MURRAY STREET WARRNAMBOOL, WARRNAMBOOL WEST      3280   
3         20 HICKFORD PDE WARRNAMBOOL, WARRNAMBOOL WEST      3280   
4     2/29 GLADSTONE STREET WARRNAMBOOL, WARRNAMBOOL...      3280   
...                                                 ...       ...   
1535                 LOT 2, 49 WALLACE STREET, MEREDITH      3333   
1536                         5490 MIDLAND HWY, MEREDITH      3333   
1537                             7 BARKERS LANE, ELAINE      3334   
1538                    324 ELAINE-EGERTON ROAD, ELAINE      3334   
1539                              PEARSONS ROAD, ELAINE      3334   

                                                      2                 3  \
0                                             cost_text     property_type   
1                

In [7]:
import random
from collections import defaultdict
import time

 
property_metadata = defaultdict(dict)
# local_ip = '45.79.208.64:44554'
# current_ip = changeIP(local_ip) 

for i in range(2052,2418):
    # sleep for 0-1 seconds
    time.sleep(random.random())
    
    headers = {'User-Agent':random.choice(user_agents)}  
    
   
    bs_object = BeautifulSoup(requests.get(url_links.iloc[i]["links"],headers = headers).text, "html.parser")
    
    '''
    # if 403 happened, changed current ip address 
    while bs_object.find("head").find("title").text == "403 Forbidden":
        current_ip = changeIP(current_ip)
        bs_object = BeautifulSoup(requests.get(row["links"],headers = headers, proxies={"http": current_ip, "https": current_ip}).text, "html.parser")
    '''    
        
    postcodes = url_links.iloc[i]["postcode"]
    bs_element = []
    if bs_object.find_all('div', {'class':'property odd clearfix'}) is not None: 
        bs_element.append(bs_object.find_all('div', {'class':'property odd clearfix'}))
    
    if bs_object.find_all('div', {'class':'property even clearfix'}) is not None:
        bs_element.append(bs_object.find_all('div', {'class':'property even clearfix'}))
    
    if len(bs_element) > 0:    
        for bs in bs_element:     
            for prop in bs:
                # looks for the header class to get property name
                name = prop \
                    .find("h2", {"class": "address"}) \
                    .text
                property_metadata[name]['name'] = prop \
                    .find("h2", {"class": "address"}) \
                    .text
                property_metadata[name]['postcode'] = postcodes

                cost = []     
                for li in prop.find("section", {"class": "grid-100 historical-price"}).find("ul"):              
                    cost_text = li.text
                    cost.append(cost_text)
                if " " in cost:
                    cost.remove(" ")
                property_metadata[name]['cost_text'] = cost

                if prop.find("p",{"class":"property-meta type"}) is not None:
                    property_metadata[name]['property_type'] = prop.find("p",{"class":"property-meta type"}).text

                if prop.find("p",{"class":"property-meta bed"}) is not None:
                    property_metadata[name]['bed'] = prop.find("p",{"class":"property-meta bed"}).text

                if prop.find("p",{"class":"property-meta bath"}) is not None:
                    property_metadata[name]['bath'] = prop.find("p",{"class":"property-meta bath"}).text

                if prop.find("p",{"class":"property-meta car"}) is not None:
                    property_metadata[name]['car'] = prop.find("p",{"class":"property-meta car"}).text

                property_metadata[name]['latitude'] = prop["data-lat"]  
                property_metadata[name]['longitude'] = prop["data-lng"]

# output to csv
pd.DataFrame(property_metadata).to_csv("../data/raw/row_2051_2418.csv")